# Import package

In [38]:
#Install module
!pip install tqdm


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
#Import package
import os
import numpy as np
import pandas as pd
from datetime import datetime
from pandas_datareader import data as web
import requests
from tqdm import tqdm

# import yfinance as yf
# import lxml
# from bs4 import BeautifulSoup #BeutifulSoup

# Some useful functions

In [40]:
#Get Alpha Vantage api key
def get_apikey(filename: str):
    with open(filename) as f:
        api_key = f.read().strip()
    f.close
    return api_key

In [50]:
#Get daily core stock data from Alpha Vantage from 2000-01 to now
def get_daily_stockdata(symbol:str, outputsize = 'compact', datatype = 'json'):
    function = 'TIME_SERIES_DAILY'
    datatype = datatype
    outputsize = outputsize
    ticker = symbol
    alpha_vantage_apikey = get_apikey(filename= 'dist/apikey_AlphaVantage')

    url = f'https://www.alphavantage.co/query?function={function}&symbol={ticker}&outputsize={outputsize}&apikey={alpha_vantage_apikey}&datatype={datatype}'
    r = requests.get(url)
    df = pd.DataFrame.from_dict(r.json()['Time Series (Daily)'],orient='index')
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    return df

# Data Preparation

## Web Scrapping

This section will perform web scrapping to scrap all S&P500 tickers on wikipedia. https://en.wikipedia.org/wiki/List_of_S%26P_500_companies

We can simply use padnas to get the table from wiki.

In [42]:
#Store the S&P information in pandas dataframe
wiki_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#Selected_changes_to_the_list_of_S&P_500_components"
tickers_df_list = pd.read_html(wiki_url)
tickers_df = tickers_df_list[0]
tickers_df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [43]:
#Turn pandas dataframe to the list
tickers = tickers_df['Symbol'].values.tolist()
print(len(tickers))

503


## Download the financial data

In [51]:
#Download the financial data from Alpha vantage
ticker = 'AAPL'
AAPL = get_daily_stockdata(ticker,outputsize='full')
print(AAPL)

                Open      High       Low     Close    Volume
2023-07-28  194.6700  196.6260  194.1400  195.8300  48291443
2023-07-27  196.0200  197.2000  192.5500  193.2200  47460180
2023-07-26  193.6700  195.6400  193.3200  194.5000  47471868
2023-07-25  193.3300  194.4400  192.9150  193.6200  37283201
2023-07-24  193.4100  194.9100  192.2500  192.7500  45505097
...              ...       ...       ...       ...       ...
1999-11-05   84.6200   88.3700   84.0000   88.3100   3721500
1999-11-04   82.0600   85.3700   80.6200   83.6200   3384700
1999-11-03   81.6200   83.2500   81.0000   81.5000   2932700
1999-11-02   78.0000   81.6900   77.3100   80.2500   3564600
1999-11-01   80.0000   80.6900   77.3700   77.6200   2487300

[5973 rows x 5 columns]
